### This notebook uses a breathing actuation sequence that computes triggers inflation deflation based upon an average inhale/exhale duration and a scale factor

In [1]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import biofeatures
import threading
import numpy as np
import time
from random import uniform

In [2]:
def data_handler(unused_addr, args, data1, data2, data3, data4, data5, data6): #BITalino ServerBIT format
    global ecg_data, resp_data
    global osc_client
        
    B = args[0]
    HR = args[1]
    
    # BITalino ServerBIT
    ecg_data.append(float(np.random.rand(1)))
    resp_data.append(float(data6))
        
    if B.is_warmed_up:
        B.set_data(resp_data[-B.buffer_length:])
    
    if HR.is_warmed_up:
        HR.set_data(ecg_data[-HR.buffer_length:])


In [3]:
def riot_data_handler(unused_addr, args, *values): #R-IoT 
    global ecg_data, resp_data
    global osc_client
    
    B = args[0]
    HR = args[1]
    
#     ecg_data.append(float(values[13]))
    ecg_data.append(np.random.rand(1)[0])
    resp_data.append(float(values[12]))
        
    if B.is_warmed_up:
        B.set_data(resp_data[-B.buffer_length:])
    
    if HR.is_warmed_up:
        HR.set_data(ecg_data[-HR.buffer_length:])
    


In [4]:
def warmup(B, resp_data, HR, ecg_data):
    """Function to launch once the system has warmed up.
    Sets the  data initially and launches a recursive update of features.
    Parameters
    ----------
    B: object containing breathing data and features
    resp_data: respiration signal data to set
    HR: object containing ECG data and features
    ecg_data: ecg signal data to set
    """
    
    if not B.is_warmed_up:
        print("Breathing WARMUP")
        B.set_data(resp_data[-B.buffer_length:])
        B.is_warmed_up = True
        B.update_loop()
        
    if not HR.is_warmed_up:
        print("ECG WARMUP")
        HR.set_data(ecg_data[-HR.buffer_length:])
        HR.is_warmed_up = True
        HR.update_loop()
        
#         act_launcher(B, resp_data, HR, ecg_data) # No thread based

        timer_act = threading.Timer(0.1, act_launcher, [B, resp_data, HR, ecg_data] ) # Thread based
        timer_act.start() # Thread based
        
#     breathing_factor = B.breathing_factor
    
#     if B.routine == "None":
#         print("n-----")

#     elif B.routine == "breathing_biofeedback":
#         timer_actuation1 = threading.Timer(0.1, breathing_biofeedback, ["1", B, True] )
#         timer_actuation1.start()
    
#         timer_actuation2 = threading.Timer(0.1, breathing_biofeedback, ["2", B, True] )
#         #timer_actuation2 = threading.Timer(0.1, pulsating, ["2", True] )
#         timer_actuation2.start()

#     elif B.routine == "pulsating":
#         pass

In [5]:
def act_launcher(B, resp_data, HR, ecg_data):
    global actuation_flag
    
    if B.routine == "None":
        print("n-------")

    elif B.routine == "breathing_biofeedback":
        breathing_factor = B.breathing_factor
        timer_actuation1 = threading.Timer(0.1, breathing_biofeedback, ["1", B, True, False, 1] )
        timer_actuation1.start()
    
        timer_actuation2 = threading.Timer(0.1, breathing_biofeedback, ["2", B, True, False, 1] )
        #timer_actuation2 = threading.Timer(0.1, pulsating, ["2", True] )
        timer_actuation2.start()

    elif B.routine == "pulsating":
        timer_actuation1 = threading.Timer(0.1, pulsating, ["1", True] )
        timer_actuation1.start()
    
        timer_actuation2 = threading.Timer(0.1, pulsating, ["2", True] )
        timer_actuation2.start()        

    elif B.routine == "random_inflation":
        timer_actuation1 = threading.Timer(0.1, random_inflation , ["1", 2, 5, True] )
        timer_actuation1.start()
    
        timer_actuation2 = threading.Timer(0.1, random_inflation, ["1", 2, 5, True] )
        timer_actuation2.start()        
    
    elif B.routine == "async_intervals":
        timer_actuation = threading.Timer(0.1, async_intervals, ["1", "2", 3, 3, True] )
        timer_actuation.start()

    elif B.routine == "async_breathing":
        timer_actuation = threading.Timer(0.1, async_breathing, ["1", "2", B, True, False, 1] )
        timer_actuation.start()    
    
    elif B.routine == "deflate3":
        timer_actuation1 = threading.Timer(0.1, deflate, ["1", 3] )
        timer_actuation1.start()
        timer_actuation1 = threading.Timer(0.1, deflate, ["2", 3] )
        timer_actuation1.start()
        
    elif B.routine == "inflate3":
        timer_actuation1 = threading.Timer(0.1, warmup_inflate, ["1", 3] )
        timer_actuation1.start()
        timer_actuation1 = threading.Timer(0.1, warmup_inflate, ["2", 3] )
        timer_actuation1.start()
        

In [6]:
def warmup_inflate(act_id, duration):
    global osc_client
    
    osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
    time.sleep(duration)
    osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)


In [7]:
def deflate(act_id, duration):
    global osc_client
    
    osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
    time.sleep(duration)
    osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)


### Different actuation sequences

In [8]:
# computes inhale and exhale interval duration as average duration of breath * breathing_factor 
def breathing_biofeedback(act_id, B, inflate, hold, hold_time):
    global osc_client 
    global actuation_flag
    
    if not actuation_flag or B.routine != "breathing_biofeedback":
        return

#     if not actuation_flag:
#         return
    
    breathing_factor = B.breathing_factor
#     print("BrF: ", breathing_factor) 

    if hold and inflate:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)
        timer_in = threading.Timer(hold_time, breathing_biofeedback, [act_id, B, True, False, hold_time])
        timer_in.start()
        
    if hold and not inflate: 
        osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)
        timer_in = threading.Timer(hold_time, breathing_biofeedback, [act_id, B, False, False, hold_time])
        timer_in.start()
    
    if not hold and inflate:
        print("inhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        timer_exhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, breathing_biofeedback, [act_id, B, False, True, hold_time])
        timer_exhale.start()
        
        for i in range(1,10):
            osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)

    if not hold and not inflate:
        print("exhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, breathing_biofeedback, [act_id, B, True, True, hold_time])
        timer_inhale.start()
        
        for i in range(1,10):
            osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)


In [2]:
# computes inhale and exhale interval duration as average duration of breath * breathing_factor 
def breathing_biofeedback(act_id, B, inflate, hold):
    global osc_client 
    global actuation_flag
    
    if not actuation_flag or B.routine != "breathing_biofeedback":
        return

#     if not actuation_flag:
#         return
    
    breathing_factor = B.breathing_factor
#     print("BrF: ", breathing_factor) 

    if hold and inflate:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)
        timer_in = threading.Timer(B.features['avg_inhale'] * B.hold_time, breathing_biofeedback, [act_id, B, True, False])
        timer_in.start()
        
    if hold and not inflate: 
        osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)
        timer_in = threading.Timer(B.features['avg_exhale'] * B.hold_time, breathing_biofeedback, [act_id, B, False, False])
        timer_in.start()
    
    if not hold and inflate:
        print("inhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        timer_exhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, breathing_biofeedback, [act_id, B, False, True])
        timer_exhale.start()
        
        for i in range(1,10):
            osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)

    if not hold and not inflate:
        print("exhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, breathing_biofeedback, [act_id, B, True, True])
        timer_inhale.start()
        
        for i in range(1,10):
            osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)



In [9]:
# specify inhale and exhale interval duration in seconds
def breathing_intervals(act_id, inhale_duration, exhale_duration, inflate):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    if inflate:
        print("inhale: ", inhale_duration)
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        timer_exhale = threading.Timer(inhale_duration, breathing_intervals, [act_id, inhale_duration, exhale_duration, False])
        timer_exhale.start()

    else:
        print("exhale: ", exhale_duration)
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inhale = threading.Timer(exhale_duration, breathing_intervals, [act_id, inhale_duration, exhale_duration, True])
        timer_inhale.start()

In [10]:
def pulsating(act_id,inflate):
    global osc_client
    global actuation_flag
    
    if not actuation_flag or B.routine != "pulsating":
        return
    
    if inflate:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 50.0)
        timer_deflate = threading.Timer(3, pulsating, [act_id, False])
        timer_deflate.start()

    else:
        osc_client.send_message("/actuator/" + act_id + "/inflate", -50.0)
        timer_inflate = threading.Timer(3, pulsating, [act_id, True])
        timer_inflate.start()

In [11]:
def random_inflation(act_id, lower, upper, inflate):
    global osc_client, actuation_flag
    
    if not actuation_flag or B.routine != "random_inflation":        
        return
    
    int_length = uniform(lower,upper)
    
    if inflate:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        timer_deflate = threading.Timer(int_length, random_inflation, [act_id, lower, upper, False])
        timer_deflate.start()

    else:
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inflate = threading.Timer(int_length, random_inflation, [act_id, lower, upper, True])
        timer_inflate.start()

In [12]:
def async_intervals(act_first, act_snd, inhale_dur, exhale_dur, inflate_first):
    global osc_client, actuation_flag
    
    if not actuation_flag or B.routine != "async_intervals":
        return
    
    if inflate_first:
        print("inhale: ", inhale_dur)
        osc_client.send_message("/actuator/" + act_first + "/inflate", 100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", -100.0)
        timer_inhale = threading.Timer(inhale_dur, async_intervals, [act_first, act_snd, inhale_dur, exhale_dur, False])
        timer_inhale.start()

    else:
        print("exhale: ", exhale_dur)
        osc_client.send_message("/actuator/" + act_first + "/inflate", -100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", 100.0)
        timer_exhale = threading.Timer(exhale_dur, async_intervals, [act_first, act_snd, inhale_dur, exhale_dur, True])
        timer_exhale.start()

In [13]:
def async_breathing(act_first, act_snd, B, inflate, hold, hold_time):
    global osc_client, actuation_flag
    
    if not actuation_flag or B.routine != "async_breathing":
        return

    breathing_factor = B.breathing_factor
    
    if inflate and hold:
        osc_client.send_message("/actuator/" + act_first + "/inflate", 0.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", 0.0)
        timer_in = threading.Timer(hold_time, async_breathing, [act_first, act_snd, B, True, False, hold_time])
        timer_in.start()
        
    if not inflate and hold:
        osc_client.send_message("/actuator/" + act_first + "/inflate", 0.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", 0.0)
        timer_in = threading.Timer(hold_time, async_breathing, [act_first, act_snd, B, False, False, hold_time])
        timer_in.start()
        
    if inflate and not hold:
        print("inhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_first + "/inflate", 100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", -100.0)
        timer_exhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, async_breathing, [act_first, act_snd, B, False, True, hold_time])
        timer_exhale.start()

    if not inflate and not hold:
        print("exhale: ", B.features['avg_exhale'])
        osc_client.send_message("/actuator/" + act_first + "/inflate", -100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", 100.0)
        timer_inhale = threading.Timer(B.features['avg_exhale'] * breathing_factor, async_breathing, [act_first, act_snd, B, True, True, hold_time])
        timer_inhale.start()

In [14]:
def async_breathing(act_first, act_snd, B, inflate, hold):
    global osc_client, actuation_flag
    
    if not actuation_flag or B.routine != "async_breathing":
        return

    breathing_factor = B.breathing_factor
    
    if inflate and hold:
        osc_client.send_message("/actuator/" + act_first + "/inflate", 0.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", 0.0)
        timer_in = threading.Timer(B.features['avg_inhale'] * B.hold_time, async_breathing, [act_first, act_snd, B, True, False])
        timer_in.start()
        
    if not inflate and hold:
        osc_client.send_message("/actuator/" + act_first + "/inflate", 0.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", 0.0)
        timer_in = threading.Timer(B.features['avg_exhale'] * B.hold_time, async_breathing, [act_first, act_snd, B, False, False])
        timer_in.start()
        
    if inflate and not hold:
        print("inhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_first + "/inflate", 100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", -100.0)
        timer_exhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, async_breathing, [act_first, act_snd, B, False, True])
        timer_exhale.start()

    if not inflate and not hold:
        print("exhale: ", B.features['avg_exhale'])
        osc_client.send_message("/actuator/" + act_first + "/inflate", -100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", 100.0)
        timer_inhale = threading.Timer(B.features['avg_exhale'] * breathing_factor, async_breathing, [act_first, act_snd, B, True, True])
        timer_inhale.start()

In [15]:
# computes degree of inflation/deflation based on ECG features

# TODO: how to compute the inflation/deflation value?

def heartrate_actuation(act_id, HR, timer_interval):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    current_trend = HR.current_trends["hr_mean"]
    #current_trend = HR.current_trends["rmssd"]
    #current_trend = HR.current_trends["LF/HF ratio"]
    
    current_feature = HR.features[-1]["hr_mean"]
    #current_feature = HR.features[-1]["rmssd"]
    #current_feature = HR.features[-1]["LF/HF ratio"]
    
    if current_trend > 0:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        #osc_client.send_message("/actuator/" + act_id + "/inflate", current_trend * 1000/current_feature)
        timer_exhale = threading.Timer(timer_interval, heartrate_actuation, [act_id, HR, timer_interval])
        timer_exhale.start()

    else:
        #osc_client.send_message("/actuator/inflate", current_trend * 1000/current_feature)
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inhale = threading.Timer(timer_interval, heartrate_actuation, [act_id, HR, timer_interval])
        timer_inhale.start()

#### Set up configuration: IP (BITalino, R-IoT or Node-RED), port and OSC address pattern

In [16]:
def hold_down():
    global actuation_flag
    a = 0
    return

In [17]:
def conf_handler(unused_addr, args, *values): #Conf 

    global ecg_data, resp_data
    global actuation_flag
    global osc_client
    
    B = args[0]
    HR = args[1]   

    print("----------------- ACTUATION CHANGE ------------------------")
    if type(values[0]) == str:
        if B.routine != values[0]:
            B.routine = values[0]

        if B.routine == "None":
            actuation_flag = False
            val = 0.0
            osc_client.send_message("/actuator/1/inflate", val)
            osc_client.send_message("/actuator/2/inflate", val)

        else:
            actuation_flag = True
           
                
            print(B.routine)
            timer_act = threading.Timer(3, act_launcher, [B, resp_data, HR, ecg_data] )
            
            timer_act.start()
        
    else: 
        B.breathing_factor = np.round(values[0],1)
        print("BrF val:", B.breathing_factor)

In [42]:
# Definitions
bitalino_ip = '192.168.0.100'
bitalino_port = 8000
bitalino_srate = 100

riot_ip = "192.168.0.100"
riot_port = 31000 
riot_srate = 200

# riot_ip = "192.168.1.40"
# riot_port = 12000 

#actuator_ip = '192.168.0.110'
actuator_ip = '192.168.0.100'
actuator_port = 32000

riot_address = "/0/raw"
bitalino_address = "/0/bitalino"


###### CONFIGURATION 
config_address = riot_address
config_ip = riot_ip
config_port = riot_port
srate = riot_srate 
C = {"config_address": config_address, "config_ip": config_ip, "config_port": config_port, "srate": srate}

osc_client = SimpleUDPClient(actuator_ip, actuator_port) 

actuation_flag = True

ecg_data = []
resp_data = []


B = biofeatures.breathing(data = np.ones(10), buffer_length=2000, srate=srate, breathing_factor = 1.6, hold_factor = 0.25)
HRV = biofeatures.hrv(data = np.ones(10), buffer_length=2000, srate=srate)
B.routine = "breathing_biofeedback" 


my_dispatcher = dispatcher.Dispatcher()
my_dispatcher.map(config_address, riot_data_handler, B, HRV)
my_dispatcher.map("/brf/", conf_handler, B, HRV)


server = osc_server.ThreadingOSCUDPServer((config_ip, config_port), my_dispatcher)
print (C)
print("Serving on {}".format(server.server_address))

# Warmup
warmup_t = 10
timer_warmup = threading.Timer(warmup_t, warmup, [B,resp_data,HRV,ecg_data])
timer_warmup.start()

try: 
    server.serve_forever()
except KeyboardInterrupt:
    B.update_data_flag = False
    HRV.update_data_flag = False
    actuation_flag = False
    server.server_close()
    osc_client.send_message("/actuator/1/inflate", 0.0)
    osc_client.send_message("/actuator/2/inflate", 0.0)
except:
    raise

{'config_address': '/0/raw', 'config_ip': '192.168.0.100', 'config_port': 31000, 'srate': 200}
Serving on ('192.168.0.100', 31000)
Breathing WARMUP
{'breath_avg_len': 3.33, 'inhale_duration': 5.34, 'avg_inhale': 2.67, 'exhale_duration': 4.64, 'avg_exhale': 1.55, 'inhale_exhale_ratio': 1.15}
ECG WARMUP
[{'nni_mean': 428.86, 'hr_mean': 160.51, 'hr_std': 61.58, 'rmssd': 232.44, 'lf': 4042.81, 'hf': 7509.28, 'LF/HF ratio': 0.54}]
inhale: inhale:  2.67
 2.67
{'breath_avg_len': 2.5, 'inhale_duration': 5.71, 'avg_inhale': 1.43, 'exhale_duration': 4.28, 'avg_exhale': 1.43, 'inhale_exhale_ratio': 1.33}
[{'nni_mean': 428.86, 'hr_mean': 160.51, 'hr_std': 61.58, 'rmssd': 232.44, 'lf': 4042.81, 'hf': 7509.28, 'LF/HF ratio': 0.54}, {'nni_mean': -10.38, 'hr_mean': 160.41, 'hr_std': 72.84, 'rmssd': 2124.85, 'lf': 64341.36, 'hf': 9741.89, 'LF/HF ratio': 6.6}]
{'breath_avg_len': 2.5, 'inhale_duration': 5.85, 'avg_inhale': 1.46, 'exhale_duration': 4.14, 'avg_exhale': 1.38, 'inhale_exhale_ratio': 1.41}
[{

[{'nni_mean': -19.19, 'hr_mean': 180.71, 'hr_std': 69.12, 'rmssd': 1958.11, 'lf': 57715.01, 'hf': 11543.16, 'LF/HF ratio': 5.0}, {'nni_mean': -16.26, 'hr_mean': 181.56, 'hr_std': 67.9, 'rmssd': 1992.18, 'lf': 59836.55, 'hf': 12655.92, 'LF/HF ratio': 4.73}, {'nni_mean': 345.19, 'hr_mean': 192.34, 'hr_std': 56.17, 'rmssd': 193.7, 'lf': 1759.98, 'hf': 4698.73, 'LF/HF ratio': 0.37}, {'nni_mean': 352.69, 'hr_mean': 184.63, 'hr_std': 51.24, 'rmssd': 166.72, 'lf': 1525.77, 'hf': 3850.74, 'LF/HF ratio': 0.4}, {'nni_mean': 347.96, 'hr_mean': 188.25, 'hr_std': 54.46, 'rmssd': 164.07, 'lf': 1654.18, 'hf': 3277.29, 'LF/HF ratio': 0.5}]
{'breath_avg_len': 3.33, 'inhale_duration': 5.9, 'avg_inhale': 1.97, 'exhale_duration': 4.1, 'avg_exhale': 2.05, 'inhale_exhale_ratio': 1.44}
{'breath_avg_len': 3.33, 'inhale_duration': 5.06, 'avg_inhale': 1.69, 'exhale_duration': 4.93, 'avg_exhale': 1.64, 'inhale_exhale_ratio': 1.02}
[{'nni_mean': -16.26, 'hr_mean': 181.56, 'hr_std': 67.9, 'rmssd': 1992.18, 'lf': 5

[{'nni_mean': 311.0, 'hr_mean': 205.46, 'hr_std': 50.11, 'rmssd': 129.1, 'lf': 126.01, 'hf': 439.61, 'LF/HF ratio': 0.29}, {'nni_mean': 317.24, 'hr_mean': 201.64, 'hr_std': 49.72, 'rmssd': 131.29, 'lf': 292.35, 'hf': 468.06, 'LF/HF ratio': 0.62}, {'nni_mean': 328.57, 'hr_mean': 194.78, 'hr_std': 49.34, 'rmssd': 138.02, 'lf': 43.41, 'hf': 580.06, 'LF/HF ratio': 0.07}, {'nni_mean': 333.15, 'hr_mean': 192.65, 'hr_std': 50.07, 'rmssd': 136.69, 'lf': 98.9, 'hf': 935.94, 'LF/HF ratio': 0.11}, {'nni_mean': -10.35, 'hr_mean': 169.83, 'hr_std': 62.95, 'rmssd': 1999.31, 'lf': 56307.21, 'hf': 13292.99, 'LF/HF ratio': 4.24}]
{'breath_avg_len': 3.33, 'inhale_duration': 4.7, 'avg_inhale': 1.57, 'exhale_duration': 5.28, 'avg_exhale': 1.76, 'inhale_exhale_ratio': 0.89}
[{'nni_mean': 317.24, 'hr_mean': 201.64, 'hr_std': 49.72, 'rmssd': 131.29, 'lf': 292.35, 'hf': 468.06, 'LF/HF ratio': 0.62}, {'nni_mean': 328.57, 'hr_mean': 194.78, 'hr_std': 49.34, 'rmssd': 138.02, 'lf': 43.41, 'hf': 580.06, 'LF/HF rat

[{'nni_mean': -24.38, 'hr_mean': 181.25, 'hr_std': 72.91, 'rmssd': 1969.51, 'lf': 64300.5, 'hf': 9509.24, 'LF/HF ratio': 6.76}, {'nni_mean': 438.0, 'hr_mean': 156.06, 'hr_std': 58.26, 'rmssd': 239.85, 'lf': 332.02, 'hf': 7784.46, 'LF/HF ratio': 0.04}, {'nni_mean': 411.36, 'hr_mean': 165.09, 'hr_std': 59.76, 'rmssd': 223.62, 'lf': 1395.21, 'hf': 2841.56, 'LF/HF ratio': 0.49}, {'nni_mean': 370.19, 'hr_mean': 180.19, 'hr_std': 58.46, 'rmssd': 175.61, 'lf': 762.29, 'hf': 1446.05, 'LF/HF ratio': 0.53}, {'nni_mean': 441.05, 'hr_mean': 152.97, 'hr_std': 53.6, 'rmssd': 211.72, 'lf': 509.14, 'hf': 14438.06, 'LF/HF ratio': 0.04}]
{'breath_avg_len': 3.33, 'inhale_duration': 4.64, 'avg_inhale': 2.32, 'exhale_duration': 5.35, 'avg_exhale': 1.78, 'inhale_exhale_ratio': 0.87}
[{'nni_mean': 438.0, 'hr_mean': 156.06, 'hr_std': 58.26, 'rmssd': 239.85, 'lf': 332.02, 'hf': 7784.46, 'LF/HF ratio': 0.04}, {'nni_mean': 411.36, 'hr_mean': 165.09, 'hr_std': 59.76, 'rmssd': 223.62, 'lf': 1395.21, 'hf': 2841.56,

exhale:  1.96
[{'nni_mean': -16.16, 'hr_mean': 167.61, 'hr_std': 69.19, 'rmssd': 2076.28, 'lf': 49916.31, 'hf': 11029.21, 'LF/HF ratio': 4.53}, {'nni_mean': -22.76, 'hr_mean': 170.69, 'hr_std': 71.81, 'rmssd': 2049.81, 'lf': 40199.77, 'hf': 9188.38, 'LF/HF ratio': 4.38}, {'nni_mean': 395.0, 'hr_mean': 170.95, 'hr_std': 59.06, 'rmssd': 224.59, 'lf': 1122.3, 'hf': 5886.42, 'LF/HF ratio': 0.19}, {'nni_mean': 392.17, 'hr_mean': 172.71, 'hr_std': 59.74, 'rmssd': 229.22, 'lf': 1177.61, 'hf': 6182.86, 'LF/HF ratio': 0.19}, {'nni_mean': 396.74, 'hr_mean': 171.88, 'hr_std': 60.71, 'rmssd': 229.22, 'lf': 1137.26, 'hf': 7118.58, 'LF/HF ratio': 0.16}]
{'breath_avg_len': 3.33, 'inhale_duration': 3.86, 'avg_inhale': 1.93, 'exhale_duration': 6.12, 'avg_exhale': 2.04, 'inhale_exhale_ratio': 0.63}
exhale:  1.93
[{'nni_mean': -22.76, 'hr_mean': 170.69, 'hr_std': 71.81, 'rmssd': 2049.81, 'lf': 40199.77, 'hf': 9188.38, 'LF/HF ratio': 4.38}, {'nni_mean': 395.0, 'hr_mean': 170.95, 'hr_std': 59.06, 'rmssd': 

...

[{'nni_mean': 365.96, 'hr_mean': 182.3, 'hr_std': 52.16, 'rmssd': 208.33, 'lf': 1645.38, 'hf': 1817.78, 'LF/HF ratio': 0.91}, {'nni_mean': 344.81, 'hr_mean': 186.5, 'hr_std': 47.74, 'rmssd': 144.47, 'lf': 188.06, 'hf': 462.7, 'LF/HF ratio': 0.41}, {'nni_mean': -32.25, 'hr_mean': 164.54, 'hr_std': 64.05, 'rmssd': 2096.85, 'lf': 3288.69, 'hf': 9920.18, 'LF/HF ratio': 0.33}, {'nni_mean': -24.54, 'hr_mean': 164.94, 'hr_std': 58.79, 'rmssd': 2004.3, 'lf': 49797.7, 'hf': 6715.85, 'LF/HF ratio': 7.41}, {'nni_mean': -11.07, 'hr_mean': 171.45, 'hr_std': 58.63, 'rmssd': 1984.13, 'lf': 48140.94, 'hf': 6184.28, 'LF/HF ratio': 7.78}]
{'breath_avg_len': 3.33, 'inhale_duration': 5.28, 'avg_inhale': 1.76, 'exhale_duration': 4.71, 'avg_exhale': 2.36, 'inhale_exhale_ratio': 1.12}
exhale:  1.76
[{'nni_mean': 344.81, 'hr_mean': 186.5, 'hr_std': 47.74, 'rmssd': 144.47, 'lf': 188.06, 'hf': 462.7, 'LF/HF ratio': 0.41}, {'nni_mean': -32.25, 'hr_mean': 164.54, 'hr_std': 64.05, 'rmssd': 2096.85, 'lf': 3288.69, 

[{'nni_mean': 410.68, 'hr_mean': 172.48, 'hr_std': 70.22, 'rmssd': 217.46, 'lf': 1372.73, 'hf': 4751.53, 'LF/HF ratio': 0.29}, {'nni_mean': 413.7, 'hr_mean': 170.39, 'hr_std': 69.32, 'rmssd': 219.69, 'lf': 1739.13, 'hf': 4332.57, 'LF/HF ratio': 0.4}, {'nni_mean': 390.0, 'hr_mean': 178.92, 'hr_std': 69.13, 'rmssd': 200.81, 'lf': 2583.51, 'hf': 3962.37, 'LF/HF ratio': 0.65}, {'nni_mean': -20.76, 'hr_mean': 174.88, 'hr_std': 78.66, 'rmssd': 2054.77, 'lf': 68380.93, 'hf': 8258.44, 'LF/HF ratio': 8.28}, {'nni_mean': 387.39, 'hr_mean': 181.5, 'hr_std': 70.84, 'rmssd': 192.82, 'lf': 3169.86, 'hf': 3640.58, 'LF/HF ratio': 0.87}]
{'breath_avg_len': 5.0, 'inhale_duration': 3.44, 'avg_inhale': 3.44, 'exhale_duration': 6.55, 'avg_exhale': 3.28, 'inhale_exhale_ratio': 0.53}
[{'nni_mean': 413.7, 'hr_mean': 170.39, 'hr_std': 69.32, 'rmssd': 219.69, 'lf': 1739.13, 'hf': 4332.57, 'LF/HF ratio': 0.4}, {'nni_mean': 390.0, 'hr_mean': 178.92, 'hr_std': 69.13, 'rmssd': 200.81, 'lf': 2583.51, 'hf': 3962.37, 

[{'nni_mean': 371.4, 'hr_mean': 182.88, 'hr_std': 65.17, 'rmssd': 178.98, 'lf': 2989.11, 'hf': 1984.56, 'LF/HF ratio': 1.51}, {'nni_mean': 364.04, 'hr_mean': 187.65, 'hr_std': 67.36, 'rmssd': 174.81, 'lf': 2184.66, 'hf': 1825.18, 'LF/HF ratio': 1.2}, {'nni_mean': 359.23, 'hr_mean': 189.33, 'hr_std': 66.26, 'rmssd': 174.88, 'lf': 1976.91, 'hf': 2122.52, 'LF/HF ratio': 0.93}, {'nni_mean': -27.56, 'hr_mean': 185.49, 'hr_std': 73.09, 'rmssd': 2009.9, 'lf': 76929.86, 'hf': 7956.14, 'LF/HF ratio': 9.67}, {'nni_mean': 361.4, 'hr_mean': 187.78, 'hr_std': 64.72, 'rmssd': 161.53, 'lf': 2415.01, 'hf': 3233.38, 'LF/HF ratio': 0.75}]
{'breath_avg_len': 5.0, 'inhale_duration': 5.36, 'avg_inhale': 2.68, 'exhale_duration': 4.63, 'avg_exhale': 2.32, 'inhale_exhale_ratio': 1.16}
[{'nni_mean': 364.04, 'hr_mean': 187.65, 'hr_std': 67.36, 'rmssd': 174.81, 'lf': 2184.66, 'hf': 1825.18, 'LF/HF ratio': 1.2}, {'nni_mean': 359.23, 'hr_mean': 189.33, 'hr_std': 66.26, 'rmssd': 174.88, 'lf': 1976.91, 'hf': 2122.52

[{'nni_mean': -22.96, 'hr_mean': 165.59, 'hr_std': 64.1, 'rmssd': 2094.59, 'lf': 72324.08, 'hf': 13986.29, 'LF/HF ratio': 5.17}, {'nni_mean': 387.71, 'hr_mean': 168.33, 'hr_std': 50.54, 'rmssd': 157.82, 'lf': 369.78, 'hf': 2836.14, 'LF/HF ratio': 0.13}, {'nni_mean': 376.8, 'hr_mean': 175.38, 'hr_std': 55.56, 'rmssd': 160.21, 'lf': 267.17, 'hf': 2211.04, 'LF/HF ratio': 0.12}, {'nni_mean': 378.54, 'hr_mean': 173.83, 'hr_std': 54.54, 'rmssd': 154.79, 'lf': 482.16, 'hf': 1294.75, 'LF/HF ratio': 0.37}, {'nni_mean': 409.76, 'hr_mean': 163.83, 'hr_std': 59.5, 'rmssd': 161.85, 'lf': 644.49, 'hf': 2543.12, 'LF/HF ratio': 0.25}]
{'breath_avg_len': 3.33, 'inhale_duration': 4.86, 'avg_inhale': 2.43, 'exhale_duration': 5.12, 'avg_exhale': 1.71, 'inhale_exhale_ratio': 0.95}
exhale:  2.43
exhale:  2.43
[{'nni_mean': 387.71, 'hr_mean': 168.33, 'hr_std': 50.54, 'rmssd': 157.82, 'lf': 369.78, 'hf': 2836.14, 'LF/HF ratio': 0.13}, {'nni_mean': 376.8, 'hr_mean': 175.38, 'hr_std': 55.56, 'rmssd': 160.21, 'l

Exception in thread Thread-142413:
Traceback (most recent call last):
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/threading.py", line 1178, in run
    self.function(*self.args, **self.kwargs)
  File "/Users/annkatrin/Documents/Uni/Master/KTH/Master Project/biofeatures/notebooks/biofeatures.py", line 268, in update_loop
    self.update_timer.start()
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/threading.py", line 848, in start
    raise RuntimeError("threads can only be started once")
RuntimeError: threads can only be started once

Exception in thread Thread-142420:
Traceback (most recent call last):
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/threading.py", line 1178, in run
    self.function(*self.args, **self.kwargs)
  File "/Users/annk

In [40]:
    osc_client.send_message("/actuator/1/inflate", 100.0)
    osc_client.send_message("/actuator/2/inflate", 100.0)


In [41]:
    osc_client.send_message("/actuator/1/inflate", 0.0)
    osc_client.send_message("/actuator/2/inflate", 0.0)



## Launch on 2 devices

In [ ]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import biofeatures
import time

# Use actuator IP
# Local Processing IP 192.168.0.100 or localhost 127.0.0.1, Arduino IP
actuator_ip = '127.0.0.1'
actuator_port = 32000
osc_client = SimpleUDPClient(actuator_ip, actuator_port) 


In [ ]:
val = 0.0
osc_client.send_message("/actuator/1/inflate", val)
osc_client.send_message("/actuator/2/inflate", val)


In [ ]:
val = 100.0
osc_client.send_message("/actuator/1/inflate", val)
osc_client.send_message("/actuator/2/inflate", val)


In [ ]:
deflate("1", 1)

In [ ]:
warmup_inflate("1", 3)

In [ ]:
warmup_inflate("2", 3)

In [ ]:
actuation_flag = True

pulsating("1", True)

In [ ]:
actuation_flag = False

In [ ]:
osc_client.send_message("/actuator/1/inflate", 0.0)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(np.linspace(0,len(resp_data)/200., len(resp_data) ), resp_data)

In [6]:
import numpy as np

np.average([1.4, 2.3, 6.4, 2.4, 4.6])

3.4200000000000004

In [7]:
round(4.234214234, 2)

4.23